In [3]:
# coding: utf-8

import datetime
import time
from pymongo import MongoClient
import pandas as pd
import numpy as np
import tweepy
import bson.objectid
import sys, traceback
import setting
import urllib.error
import urllib.request
import json

In [7]:
import twpy_app_kani
api = twpy_app_kani.api

client = MongoClient('localhost', 27017)
db = client.twipic
# subaru = db.subaru
# momoko = db.momoko
# hibiki = db.hibiki
# loco = db.loco
collection = db.godo

In [5]:
def get_the_value(keyword, **kwargs):
    res = {"exist_key":False, "value":None}
    for key in kwargs.keys():
        # print(key, kwargs[key])
        if key == keyword:
            res["exist_key"] = True
            res["value"] = kwargs[key]
        if isinstance(kwargs[key], dict):
            res_new = get_the_value(keyword, **kwargs[key])
            if res_new["exist_key"] == True: res = res_new
        # else:
    return res

In [6]:
def pic_getter(collection, keyword, min_retweets):
    print("start")

    keywords =[keyword, 'filter:media -filter:retweets min_retweets:'+ str(min_retweets)]
    search_text = ' AND '.join(keywords)
    print(search_text)
    for result_tweet in tweepy.Cursor(api.search, q=search_text, count=100, tweet_mode='extended', wait_on_rate_limit = True).items():
        exist_tweet = False
        for a in collection.find({'tweet_id':result_tweet.id_str}):
            exist_tweet = True
        if exist_tweet==False:
            tweet = {}
            tweet['tweet_id'] = result_tweet.id_str
            tweet['user_id'] = result_tweet.user.id_str
            tweet['screen_name'] = result_tweet.user.screen_name
            tweet['text'] = result_tweet.full_text
            tweet['created_at'] = result_tweet.created_at
            tweet['followers_count'] = result_tweet.user.followers_count
            tweet['friends_count'] = result_tweet.user.friends_count
            tweet['retweet_count'] = result_tweet.retweet_count
            tweet['state'] = 0
            media = get_the_value("media", **result_tweet._json)
            if media["exist_key"]:
                tweet['media'] = media["value"]
                collection.insert_one(tweet)
    print("end") 

In [2]:
idol_name = "ロコ"
text = ['#{0}生誕祭2019 OR #{0}生誕祭 OR #{0}誕生祭 OR #{0}誕生祭2019', '{} AND 誕生' ,'{} AND "おめでと"']
text = [x.format(idol_name) for x in text]
text

['#ロコ生誕祭2019 OR #ロコ生誕祭 OR #ロコ誕生祭 OR #ロコ誕生祭2019', 'ロコ AND 誕生', 'ロコ AND "おめでと"']

In [9]:
# text = ['#周防桃子生誕祭2019 OR #周防桃子生誕祭 OR #周防桃子誕生祭 OR #周防桃子誕生祭2019', '桃子 AND 誕生' ,'桃子 AND "おめでと"']
# text = ['#ロコ生誕祭2019 OR #ロコ生誕祭 OR #ロコ誕生祭 OR #ロコ誕生祭2019', '#伴田路子生誕祭2019 OR #伴田路子生誕祭 OR #伴田路子誕生祭 OR #伴田路子誕生祭2019', 'ロコ AND 誕生' ,'ロコ AND "おめでと"']
text = ['#周防桃子合同']
min_retweets = 10
res = None
for keywords in text:
    pic_getter(collection, keywords, min_retweets)

start
#周防桃子合同 AND filter:media -filter:retweets min_retweets:10
end


In [79]:
# check
print(res._json['created_at'])
media = get_the_value("media", **res._json)
for m in media["value"]:
    print(m['media_url'])

http://pbs.twimg.com/media/ESHMpbcVUAAt2ba.jpg


In [10]:
collection.find().count()

33

In [12]:
import urllib.error
import urllib.request

def download_image(url, dst_path):
    try:
        data = urllib.request.urlopen(url).read()
        with open(dst_path, mode="wb") as f:
            f.write(data)
    except urllib.error.URLError as e:
        print(e)
        
start = datetime.datetime(2020, 2, 29, 0, 0, 0)        
        
i=0
data = []
for d in collection.find({'created_at': {'$gte': start}}):
    data.append(d)

In [13]:
for (di, d) in enumerate(data):
    print(d['created_at'])
    for (mi, media) in enumerate(d['media']):
        url = '%s:orig' % media['media_url_https']
        dst_path = 'C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_{0:03d}_{1}.jpg'.format(di+1, mi+1)
        print(dst_path)
        # dst_dir = 'data/src'
        # dst_path = os.path.join(dst_dir, os.path.basename(url))
        download_image(url, dst_path)

2020-03-08 03:17:46
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_001_1.jpg
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_001_2.jpg
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_001_3.jpg
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_001_4.jpg
2020-03-07 09:35:41
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_002_1.jpg
2020-03-07 06:05:31
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_003_1.jpg
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_003_2.jpg
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_003_3.jpg
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_003_4.jpg
2020-03-07 03:35:43
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_004_1.jpg
2020-03-04 14:05:05
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_005_1.jpg
2020-03-04 04:23:33
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_006_1.jpg
2020-03-03 15:23:33
C:/Users/Kanazawa/Pictures/scrpic/momokogodo/momokododo_007_1.jpg
20

In [5]:
class Twitter_API():
    def __init__(self, key_number, dbName, collectionName):
        consumer_key = setting.key['consumer_key'][key_number]
        consumer_secret = setting.key['consumer_secret'][key_number]
        auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
        self.api = tweepy.API(auth)
        
        self.client = MongoClient('localhost', 27017)
        self.db = self.client[dbName] #DB名を設定
        self.collection = self.db.get_collection(collectionName)
        
    def pic_getter(self, keyword, min_retweets):
        print("start")

        keywords =[keyword, 'filter:media -filter:retweets min_retweets:'+ str(min_retweets)]
        search_text = ' AND '.join(keywords)
        print(search_text)
        for result_tweet in tweepy.Cursor(self.api.search, q=search_text, count=100, tweet_mode='extended', wait_on_rate_limit = True).items():
            # print('media' in result_tweet.entities)
            exist_tweet = False
            for a in self.collection.find({'tweet_id':result_tweet.id_str}):
                exist_tweet = True
            if exist_tweet==False:
                tweet = {}
                tweet['tweet_id'] = result_tweet.id_str
                tweet['user_id'] = result_tweet.user.id_str
                tweet['screen_name'] = result_tweet.user.screen_name
                tweet['text'] = result_tweet.full_text
                tweet['created_at'] = result_tweet.created_at
                tweet['followers_count'] = result_tweet.user.followers_count
                tweet['friends_count'] = result_tweet.user.friends_count
                tweet['retweet_count'] = result_tweet.retweet_count
                tweet['state'] = 0
                print('media' in result_tweet.extended_entities)
                if 'media' in result_tweet.extended_entities:
                    tweet['media'] = result_tweet.extended_entities['media']
                    # self.collection.insert_one(tweet) ###########################

        print("end")
    
    def download_image(self, url, dst_path):
        try:
            data = urllib.request.urlopen(url).read()
            with open(dst_path, mode="wb") as f:
                f.write(data)
        except urllib.error.URLError as e:
            print(e)
        

In [5]:
api = Twitter_API(1, "twipic", "subaru")
text = ['#永吉昴生誕祭2019 OR #永吉昴生誕祭 OR #永吉昴誕生祭 OR #永吉昴誕生祭2019', '昴 AND 誕生' ,'昴 AND "おめでと"']
min_retweets = 10
for keywords in text:
    api.pic_getter(keywords, min_retweets)

start
#永吉昴生誕祭2019 OR #永吉昴生誕祭 OR #永吉昴誕生祭 OR #永吉昴誕生祭2019 AND filter:media -filter:retweets min_retweets:10
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


# 以下メモ